# Connecting to VM

In [1]:
from chi import server, storage, context
import chi, os, time, datetime

context.version = "1.0" 
context.choose_project()
context.choose_site(default="KVM@TACC")

In [2]:
username = os.getenv('USER') # all exp resources will have this prefix
s = server.Server(
    f"node-persist-mlops-project2", 
    image_name="CC-Ubuntu24.04",
    flavor_name="m1.medium",
)
s.submit(idempotent=True)

Waiting for server node-persist-mlops-project2's status to become ACTIVE. This typically takes 10 minutes, but can take up to 20 minutes.


Server has moved to status ACTIVE


Attribute,node-persist-mlops-project2
Id,32f7997f-77cf-412b-8e1b-c9cd6a378a43
Status,ACTIVE
Image Name,CC-Ubuntu24.04
Flavor Name,m1.medium
Addresses,sharednet1: IP: 10.56.0.129 (v4) Type: fixed MAC: fa:16:3e:c0:ab:08 IP: 129.114.25.26 (v4) Type: floating MAC: fa:16:3e:c0:ab:08private_cloud_net_project2: IP: 192.168.1.11 (v4) Type: fixed MAC: fa:16:3e:04:67:23
Network Name,private_cloud_net_project2
Created At,2025-05-05T20:04:29Z
Keypair,id_rsa_mlops_proj
Reservation Id,None
Host Id,fa23df14d797c3d9b076bd05a5592231b73b47708a6b75ec9d0b406c


In [10]:
s.refresh()
# s.show(type="widget")

In [ ]:
s.refresh()
s.check_connectivity()

# Create Persistent Storage

In [6]:
# get the Cinder Python client configured by python-chi
cinder_client = chi.clients.cinder()

In [7]:
# list current volumes
cinder_client.volumes.list()

[<Volume: 2e363ec1-5e2b-43e1-9c42-be5adeadf205>,
 <Volume: 631c779f-9bab-4df7-8d55-9f34b173e1f2>,
 <Volume: 768917db-be7c-4fb3-b368-fac14549d382>,
 <Volume: 1725b016-f765-409a-84c8-edb3819310a2>,
 <Volume: d0c861fe-5016-4154-929d-e90bf45bf6df>,
 <Volume: b1438009-b902-4e47-9e6a-955df2ad1d78>,
 <Volume: 1a381ba1-f025-4371-9146-2efbe15f926c>,
 <Volume: b9dbf770-b4d7-47bf-beaf-e50fd18a0971>,
 <Volume: c0426918-6c3c-4a5b-b67c-b7fbce286677>,
 <Volume: dce99464-a11e-4fc1-9853-385559c03796>,
 <Volume: ec013b3f-12a7-44c8-b752-53737b2426d9>,
 <Volume: b66b20f7-cdb1-454e-a789-6c69be149faa>,
 <Volume: ada8e171-e7fd-4e8b-b8ea-83a1b67b1bf5>,
 <Volume: 1c93da13-eae6-4594-8d14-84c021d18c55>,
 <Volume: ff2bc916-fa34-4ed8-9325-b5b1acbde276>,
 <Volume: 19c615ef-584a-484c-a5e2-dc1acfa19af8>,
 <Volume: d7d03c0e-6293-4993-bbcf-a1c72b14f4d3>,
 <Volume: d5160a9b-3c25-453d-b798-1f96c1adb0ec>,
 <Volume: ed6061ca-b65e-4f86-9c7b-2e8ec93db3f1>,
 <Volume: eb6a588b-534a-416d-af10-184d030f6cbf>,
 <Volume: 583b519a-e

In [8]:
# create a volume, specifying name and size in GiB
volume = cinder_client.volumes.create(name=f"block-persist-python-project2", size = 20)
volume._info

{'id': 'e3f1235d-f473-4677-a735-9474520c3ba9',
 'status': 'creating',
 'size': 20,
 'availability_zone': 'nova',
 'created_at': '2025-05-09T20:06:51.988750',
 'updated_at': None,
 'name': 'block-persist-python-project2',
 'description': None,
 'volume_type': 'ceph-hdd',
 'snapshot_id': None,
 'source_volid': None,
 'metadata': {},
 'links': [{'rel': 'self',
   'href': 'https://kvm.tacc.chameleoncloud.org:8776/v3/89f528973fea4b3a981f9b2344e522de/volumes/e3f1235d-f473-4677-a735-9474520c3ba9'},
  {'rel': 'bookmark',
   'href': 'https://kvm.tacc.chameleoncloud.org:8776/89f528973fea4b3a981f9b2344e522de/volumes/e3f1235d-f473-4677-a735-9474520c3ba9'}],
 'user_id': '3772a2b3c4c9ef99560c8940f7b8e754f6388a2d5f9c4f42691ed173013f8d3f',
 'bootable': 'false',
 'encrypted': False,
 'replication_status': None,
 'consistencygroup_id': None,
 'multiattach': False,
 'attachments': []}

In [9]:
server_id      = chi.server.get_server(f"node-persist-mlops-project2").id

In [10]:
volume_manager = chi.nova().volumes
volume_manager.create_server_volume(server_id = s.id, volume_id = volume.id)

<Volume: e3f1235d-f473-4677-a735-9474520c3ba9>

In [ ]:
#volume_manager.delete_server_volume(server_id = s.id, volume_id = volume.id)